<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

gen_output = 0

Run date time (IST): 2023-07-05 15:51:31


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [3]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [4]:
# set start and end date
start_date = (datetime.date.today() + datetime.timedelta(days=-5*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')



In [5]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/myProspectsScrips.csv')

mypf = mypf[mypf['InPortfolio'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
stock_ids

array(['5PAISA', 'AAVAS', 'AMARAJABAT', 'AMBUJACEM', 'ASIANPAINT',
       'BAJAJFINSV', 'BAJFINANCE', 'BANDHANBNK', 'BATAINDIA', 'BAYERCROP',
       'CDSL', 'CLEAN', 'DMART', 'EMAMILTD', 'FINEORG', 'GICRE',
       'GILLETTE', 'GLAND', 'HDFCAMC', 'ICICIGI', 'INDIGOPNTS',
       'INDUSINDBK', 'INFY', 'IRCTC', 'ITBEES', 'JMFINANCIL', 'JUBLFOOD',
       'LTIM', 'LUXIND', 'METROPOLIS', 'NIACL', 'PAGEIND', 'PIDILITIND',
       'PNB', 'RAJESHEXPO', 'RELAXO', 'RELIANCE', 'ROSSARI', 'SUNTV',
       'TCS', 'TEAMLEASE', 'UJJIVANSFB', 'VAIBHAVGBL', 'VALIANTORG',
       'WIPRO'], dtype=object)

In [6]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))
    stock_df = yfin.Ticker(stock_symbol).history(period='5y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Stock'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'
                   ], axis=1, inplace=True)

    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    #current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'
    tmp = stock_prec_dev(stock_id)
    tmp['Prev_Close'] = tmp['Close'].shift(1)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
    tmp['Criteria'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]
    tmp['Strategy'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]
    #tmp['Curr_PE'] = current_pe
    df_prec_dev = df_prec_dev.append(tmp)

In [7]:
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

In [8]:
# portfolio stocks

tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df['Investment'] = tmp_df['Avg_Cost'] * tmp_df['Shares']
tmp_df['Current'] = tmp_df['Close'] * tmp_df['Shares']
tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
tmp_df['Estimated'] = tmp_df['Max'] * tmp_df['Shares']
tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Estimated P/L%'] = round((tmp_df['Estimated'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Juice Left%'] = tmp_df['Estimated P/L%'] - tmp_df['Current P/L%']

investment = round(sum(tmp_df['Avg_Cost']*tmp_df['Shares']),0)
current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
previous = round(sum(tmp_df['Prev_Close']*tmp_df['Shares']),0)

today_pnl_amount = current-previous
today_pnl_percentage = round((current-previous)*100/previous,2)

curr_pnl_amount = current-investment
curr_pnl_percentage = round((current-investment)*100/investment,2)

estimate = round(sum(tmp_df['Max']*tmp_df['Shares']),0)
est_pnl_amount = estimate-investment
est_pnl_percentage = round((estimate-investment)*100/investment,2)

tmp_df['Weightage%'] = round(tmp_df['Investment']*100/investment,2)
tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['Weightage%'],0)

print('Investment: ', investment)
print('Current: ', current)
print('Today PnL: ', today_pnl_amount)
print('Today PnL %: ', today_pnl_percentage)
print('Current PnL: ', curr_pnl_amount)
print('Current PnL %: ', curr_pnl_percentage)
print('Estimate: ', estimate)
print('Estimate PnL: ', est_pnl_amount)
print('Estimate PnL %: ', est_pnl_percentage)

qualified stocks: 45
Investment:  2994570.0
Current:  3160646.0
Today PnL:  13928.0
Today PnL %:  0.44
Current PnL:  166076.0
Current PnL %:  5.55
Estimate:  5350835.0
Estimate PnL:  2356265.0
Estimate PnL %:  78.68


In [9]:
tmp_df.sort_values(by = 'Current P/L%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
18,2294.80,HDFCAMC.NS,3518.0,2291.35,1879.66,41.0,V40,ATH,77066.06,94086.80,93945.35,144238.0,0.15,22.09,87.16,65.07,2.57,57.0
6,7838.75,BAJFINANCE.NS,7860.0,7860.45,6602.15,24.0,V40,ATH,158451.60,188130.00,188650.80,188640.0,-0.28,18.73,19.05,0.32,5.29,99.0
0,411.55,5PAISA.NS,553.0,368.95,347.06,228.0,V40N,ATH,79129.68,93833.40,84120.60,126084.0,11.55,18.58,59.34,40.76,2.64,49.0
19,1327.10,ICICIGI.NS,1613.0,1315.05,1137.80,69.0,V40,ATH,78508.20,91569.90,90738.45,111297.0,0.92,16.64,41.76,25.12,2.62,44.0
41,40.65,UJJIVANSFB.NS,57.0,40.15,34.87,1350.0,V40N,ATH,47074.50,54877.50,54202.50,76950.0,1.25,16.58,63.46,46.88,1.57,26.0
10,1196.55,CDSL.NS,1648.0,1144.15,1027.05,49.0,SS,ATH,50325.45,58630.95,56063.35,80752.0,4.58,16.50,60.46,43.96,1.68,28.0
33,58.60,PNB.NS,95.0,57.05,50.85,900.0,SS,ATH,45765.00,52740.00,51345.00,85500.0,2.72,15.24,86.82,71.58,1.53,23.0
16,5129.75,GILLETTE.NS,7529.0,5125.10,4475.85,21.0,V40,ATH,93992.85,107724.75,107627.10,158109.0,0.09,14.61,68.21,53.60,3.14,46.0
5,1613.70,BAJAJFINSV.NS,1905.0,1629.50,1454.93,52.0,V40,ATH,75656.36,83912.40,84734.00,99060.0,-0.97,10.91,30.93,20.02,2.53,28.0
1,1540.30,AAVAS.NS,3196.0,1531.75,1390.60,23.0,2T3Y,2T3Y,31983.80,35426.90,35230.25,73508.0,0.56,10.77,129.83,119.06,1.07,12.0


In [10]:
tmp_df.sort_values(by = 'Estimated P/L%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
17,1064.05,GLAND.NS,4316.0,1051.80,1225.87,55.0,2T3Y,2T3Y,67422.85,58522.75,57849.00,237380.0,1.16,-13.20,252.08,265.28,2.25,-30.0
43,542.35,VALIANTORG.NS,1851.0,537.35,542.52,144.0,2T3Y,2T3Y,78122.88,78098.40,77378.40,266544.0,0.93,-0.03,241.19,241.22,2.61,-0.0
28,1499.25,LUXIND.NS,4546.0,1501.10,1493.72,51.0,V40N,2T3Y,76179.72,76461.75,76556.10,231846.0,-0.12,0.37,204.34,203.97,2.54,1.0
42,308.55,VAIBHAVGBL.NS,966.0,309.60,317.48,144.0,2T3Y,2T3Y,45717.12,44431.20,44582.40,139104.0,-0.34,-2.81,204.27,207.08,1.53,-4.0
7,224.95,BANDHANBNK.NS,723.0,231.25,253.98,300.0,2T3Y,2T3Y,76194.00,67485.00,69375.00,216900.0,-2.72,-11.43,184.67,196.10,2.54,-29.0
30,116.15,NIACL.NS,304.0,116.40,117.07,390.0,TBD,ATH,45657.30,45298.50,45396.00,118560.0,-0.21,-0.79,159.67,160.46,1.52,-1.0
29,1461.00,METROPOLIS.NS,3440.0,1462.40,1365.98,22.0,TBD,ATH,30051.56,32142.00,32172.80,75680.0,-0.10,6.96,151.83,144.87,1.00,7.0
40,2409.50,TEAMLEASE.NS,5281.0,2452.75,2279.93,21.0,V40N,2T3Y,47878.53,50599.50,51507.75,110901.0,-1.76,5.68,131.63,125.95,1.60,9.0
1,1540.30,AAVAS.NS,3196.0,1531.75,1390.60,23.0,2T3Y,2T3Y,31983.80,35426.90,35230.25,73508.0,0.56,10.77,129.83,119.06,1.07,12.0
20,1428.25,INDIGOPNTS.NS,3112.0,1436.25,1407.73,124.0,V40N,2T3Y,174558.52,177103.00,178095.00,385888.0,-0.56,1.46,121.07,119.61,5.83,9.0


In [11]:
tmp_df.sort_values(by = 'Juice Left%', ascending=True)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
6,7838.75,BAJFINANCE.NS,7860.0,7860.45,6602.15,24.0,V40,ATH,158451.60,188130.00,188650.80,188640.0,-0.28,18.73,19.05,0.32,5.29,99.0
4,3374.40,ASIANPAINT.NS,3542.0,3347.70,3091.34,25.0,V40,ATH,77283.50,84360.00,83692.50,88550.0,0.80,9.16,14.58,5.42,2.58,24.0
36,2584.50,RELIANCE.NS,2811.0,2588.75,2467.15,31.0,V40,ATH,76481.65,80119.50,80251.25,87141.0,-0.16,4.76,13.94,9.18,2.55,12.0
32,2608.85,PIDILITIND.NS,2902.0,2570.45,2504.06,30.0,V40,ATH,75121.80,78265.50,77113.50,87060.0,1.49,4.18,15.89,11.71,2.51,10.0
39,3319.95,TCS.NS,3857.0,3308.85,3231.00,16.0,V40,ATH,51696.00,53119.20,52941.60,61712.0,0.34,2.75,19.37,16.62,1.73,5.0
5,1613.70,BAJAJFINSV.NS,1905.0,1629.50,1454.93,52.0,V40,ATH,75656.36,83912.40,84734.00,99060.0,-0.97,10.91,30.93,20.02,2.53,28.0
19,1327.10,ICICIGI.NS,1613.0,1315.05,1137.80,69.0,V40,ATH,78508.20,91569.90,90738.45,111297.0,0.92,16.64,41.76,25.12,2.62,44.0
24,31.26,ITBEES.NS,40.0,30.94,30.07,1675.0,SS,ATH,50367.25,52360.50,51824.50,67000.0,1.03,3.96,33.02,29.06,1.68,7.0
8,1658.90,BATAINDIA.NS,2173.0,1632.20,1523.62,100.0,V40,ATH,152362.00,165890.00,163220.00,217300.0,1.64,8.88,42.62,33.74,5.09,45.0
9,4581.80,BAYERCROP.NS,6002.0,4570.20,4185.24,11.0,V40N,ATH,46037.64,50399.80,50272.20,66022.0,0.25,9.48,43.41,33.93,1.54,15.0


In [12]:
tmp_df.sort_values(by = 'Weightage%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
20,1428.25,INDIGOPNTS.NS,3112.0,1436.25,1407.73,124.0,V40N,2T3Y,174558.52,177103.00,178095.00,385888.0,-0.56,1.46,121.07,119.61,5.83,9.0
6,7838.75,BAJFINANCE.NS,7860.0,7860.45,6602.15,24.0,V40,ATH,158451.60,188130.00,188650.80,188640.0,-0.28,18.73,19.05,0.32,5.29,99.0
8,1658.90,BATAINDIA.NS,2173.0,1632.20,1523.62,100.0,V40,ATH,152362.00,165890.00,163220.00,217300.0,1.64,8.88,42.62,33.74,5.09,45.0
31,36721.60,PAGEIND.NS,53549.0,37122.30,39325.03,3.0,V40,ATH,117975.09,110164.80,111366.90,160647.0,-1.08,-6.62,36.17,42.79,3.94,-26.0
16,5129.75,GILLETTE.NS,7529.0,5125.10,4475.85,21.0,V40,ATH,93992.85,107724.75,107627.10,158109.0,0.09,14.61,68.21,53.60,3.14,46.0
0,411.55,5PAISA.NS,553.0,368.95,347.06,228.0,V40N,ATH,79129.68,93833.40,84120.60,126084.0,11.55,18.58,59.34,40.76,2.64,49.0
19,1327.10,ICICIGI.NS,1613.0,1315.05,1137.80,69.0,V40,ATH,78508.20,91569.90,90738.45,111297.0,0.92,16.64,41.76,25.12,2.62,44.0
43,542.35,VALIANTORG.NS,1851.0,537.35,542.52,144.0,2T3Y,2T3Y,78122.88,78098.40,77378.40,266544.0,0.93,-0.03,241.19,241.22,2.61,-0.0
4,3374.40,ASIANPAINT.NS,3542.0,3347.70,3091.34,25.0,V40,ATH,77283.50,84360.00,83692.50,88550.0,0.80,9.16,14.58,5.42,2.58,24.0
26,493.55,JUBLFOOD.NS,903.0,499.90,471.56,164.0,SS,ATH,77335.84,80942.20,81983.60,148092.0,-1.27,4.66,91.49,86.83,2.58,12.0


In [13]:
if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  tmp_df.to_csv('/content/drive/My Drive/data/stocks/myStocks-Portfolio-Analysis.csv', index=False)

In [14]:
tmp_df.sort_values(by = 'Risk Ind', ascending=True)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
17,1064.05,GLAND.NS,4316.0,1051.80,1225.87,55.0,2T3Y,2T3Y,67422.85,58522.75,57849.00,237380.0,1.16,-13.20,252.08,265.28,2.25,-30.0
7,224.95,BANDHANBNK.NS,723.0,231.25,253.98,300.0,2T3Y,2T3Y,76194.00,67485.00,69375.00,216900.0,-2.72,-11.43,184.67,196.10,2.54,-29.0
31,36721.60,PAGEIND.NS,53549.0,37122.30,39325.03,3.0,V40,ATH,117975.09,110164.80,111366.90,160647.0,-1.08,-6.62,36.17,42.79,3.94,-26.0
34,529.25,RAJESHEXPO.NS,949.0,528.25,624.78,48.0,V40N,ATH,29989.44,25404.00,25356.00,45552.0,0.19,-15.29,51.89,67.18,1.00,-15.0
42,308.55,VAIBHAVGBL.NS,966.0,309.60,317.48,144.0,2T3Y,2T3Y,45717.12,44431.20,44582.40,139104.0,-0.34,-2.81,204.27,207.08,1.53,-4.0
11,1378.45,CLEAN.NS,2648.0,1387.65,1396.49,23.0,2T3Y,2T3Y,32119.27,31704.35,31915.95,60904.0,-0.66,-1.29,89.62,90.91,1.07,-1.0
38,443.45,SUNTV.NS,710.0,445.60,446.75,68.0,V40N,ATH,30379.00,30154.60,30300.80,48280.0,-0.48,-0.74,58.93,59.67,1.01,-1.0
23,630.60,IRCTC.NS,1164.0,628.70,633.05,120.0,SS,ATH,75966.00,75672.00,75444.00,139680.0,0.30,-0.39,83.87,84.26,2.54,-1.0
30,116.15,NIACL.NS,304.0,116.40,117.07,390.0,TBD,ATH,45657.30,45298.50,45396.00,118560.0,-0.21,-0.79,159.67,160.46,1.52,-1.0
37,843.05,ROSSARI.NS,1569.0,849.25,844.40,36.0,2T3Y,2T3Y,30398.40,30349.80,30573.00,56484.0,-0.73,-0.16,85.81,85.97,1.02,-0.0


In [15]:
print('Total profit: ',round(sum(tmp_df[tmp_df['Current P/L%'] > 0]['Current']) - sum(tmp_df[tmp_df['Current P/L%'] > 0]['Investment']),0))
print('Total loss: ',round(sum(tmp_df[tmp_df['Current P/L%'] < 0]['Current']) - sum(tmp_df[tmp_df['Current P/L%'] < 0]['Investment']),0))

Total profit:  198760.0
Total loss:  -32684.0


In [16]:
# SIP based on portfolio
tmp_df[tmp_df['Weightage%'] < 2].sort_values(by = 'Current P/L%', ascending=True)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
34,529.25,RAJESHEXPO.NS,949.0,528.25,624.78,48.0,V40N,ATH,29989.44,25404.00,25356.00,45552.0,0.19,-15.29,51.89,67.18,1.00,-15.0
42,308.55,VAIBHAVGBL.NS,966.0,309.60,317.48,144.0,2T3Y,2T3Y,45717.12,44431.20,44582.40,139104.0,-0.34,-2.81,204.27,207.08,1.53,-4.0
11,1378.45,CLEAN.NS,2648.0,1387.65,1396.49,23.0,2T3Y,2T3Y,32119.27,31704.35,31915.95,60904.0,-0.66,-1.29,89.62,90.91,1.07,-1.0
30,116.15,NIACL.NS,304.0,116.40,117.07,390.0,TBD,ATH,45657.30,45298.50,45396.00,118560.0,-0.21,-0.79,159.67,160.46,1.52,-1.0
38,443.45,SUNTV.NS,710.0,445.60,446.75,68.0,V40N,ATH,30379.00,30154.60,30300.80,48280.0,-0.48,-0.74,58.93,59.67,1.01,-1.0
37,843.05,ROSSARI.NS,1569.0,849.25,844.40,36.0,2T3Y,2T3Y,30398.40,30349.80,30573.00,56484.0,-0.73,-0.16,85.81,85.97,1.02,-0.0
13,422.20,EMAMILTD.NS,588.0,423.95,422.59,72.0,V200,ATH,30426.48,30398.40,30524.40,42336.0,-0.41,-0.09,39.14,39.23,1.02,-0.0
39,3319.95,TCS.NS,3857.0,3308.85,3231.00,16.0,V40,ATH,51696.00,53119.20,52941.60,61712.0,0.34,2.75,19.37,16.62,1.73,5.0
14,4841.85,FINEORG.NS,7196.0,4882.65,4697.43,11.0,V40N,ATH,51671.73,53260.35,53709.15,79156.0,-0.84,3.07,53.19,50.12,1.73,5.0
24,31.26,ITBEES.NS,40.0,30.94,30.07,1675.0,SS,ATH,50367.25,52360.50,51824.50,67000.0,1.03,3.96,33.02,29.06,1.68,7.0
